In [2]:
!pip install -qU xpander-sdk llama-index llama-index-llms-openai python-dotenv


In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]

In [4]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

def square(a: int) -> int:
    """Multiply a by itself and returns the result integer"""
    return a*a

square_tool = FunctionTool.from_defaults(fn=square)

In [5]:
# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4")
agent = ReActAgent.from_tools([multiply_tool, add_tool, square_tool], llm=llm, verbose=True)

In [6]:
response_gen = agent.stream_chat("What is 20+2*4? Calculate step by step")
response_gen.print_response_stream()

> Running step ba0746e5-34f8-48df-ae60-267e523f3893. Step input: What is 20+2*4? Calculate step by step
Thought: The user wants to calculate the expression 20+2*4 step by step. According to the order of operations (BIDMAS/BODMAS), multiplication should be performed before addition. So, I will first multiply 2 and 4, and then add the result to 20.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step 72e0c674-9f98-429e-9bfb-255e0beeb505. Step input: None
Thought: The multiplication of 2 and 4 is 8. Now, I will add this result to 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step 66028412-d47b-4c90-989c-d1e1c40340d9. Step input: None
 The result of the expression 20+2*4 is 28.

In [6]:
response_gen_2 = agent.stream_chat("What is the square of 20? Calculate step by step")
response_gen_2.print_response_stream()

> Running step fe530ef1-139d-497c-adef-1717474da6dc. Step input: What is the square of 20? Calculate step by step
Thought: The user wants to know the square of 20. The square of a number is calculated by multiplying the number by itself. I can use the 'square' tool to calculate this.
Action: square
Action Input: {'a': 20}
Observation: 400
> Running step 4184b18c-6fff-4bf1-86ea-64f193b25471. Step input: None
 The square of 20 is calculated by multiplying 20 by itself. So, 20 * 20 = 400. Therefore, the square of 20 is 400.

In [7]:
response_gen_3 = agent.stream_chat("What is the square of 14 + 12 squared? Calculate step by step")

> Running step 66fb880d-cfae-49b9-9de0-1a984e46a41e. Step input: What is the square of 14 + 12 squared? Calculate step by step
Thought: The user wants to calculate the square of 14 plus the square of 12. This involves two separate calculations: squaring 14 and squaring 12. I'll use the 'square' tool for both of these calculations.
Action: square
Action Input: {'a': 14}
Observation: 196
> Running step c4345cd6-023f-48fd-bf65-fd2cc7d20569. Step input: None
Thought: The square of 14 is 196. Now, I need to calculate the square of 12.
Action: square
Action Input: {'a': 12}
Observation: 144
> Running step 7fa2675d-01b9-48da-8d18-72b1fcc8b0a5. Step input: None
Thought: The square of 12 is 144. Now, I need to add these two results together to get the final answer.
Action: add
Action Input: {'a': 196, 'b': 144}
Observation: 340
> Running step eaf747eb-c117-4233-847f-13d66847dc9b. Step input: None


In [8]:
response_gen_3.print_response_stream()

 The square of 14 plus the square of 12 equals 340.

In [9]:
response_gen_4 = agent.stream_chat("What is the square of 14 + the square of 12? Calculate step by step")
response_gen_4.print_response_stream()

> Running step c37f4b0b-f0cd-4b42-8140-9f79f447a00f. Step input: What is the square of 14 + the square of 12? Calculate step by step
Thought: The user wants to know the square of 14 added to the square of 12. I will use the square tool twice to get the squares of 14 and 12, and then add the results together.
Action: square
Action Input: {'a': 14}
Observation: 196
> Running step 6743ad3b-2838-4f25-8ba5-b02855cb2412. Step input: None
Thought: The square of 14 is 196. Now I need to calculate the square of 12.
Action: square
Action Input: {'a': 12}
Observation: 144
> Running step e74ac37b-503f-45ed-b1fe-e0d4c3e1120d. Step input: None
Thought: The square of 12 is 144. Now I need to add 196 and 144 to get the final result.
Action: add
Action Input: {'a': 196, 'b': 144}
Observation: 340
> Running step 162de446-d9e3-49ab-a895-d3f92b3e2124. Step input: None
 The square of 14 plus the square of 12 equals 340.

In [3]:
from openai import OpenAI
from xpander_sdk import XpanderClient, LLMProvider
# load the client
xpander_client = XpanderClient(api_key=XPANDER_API_KEY, organization_id="")

# get the agent
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# xpander_agent.add_local_tools([multiply_tool, add_tool, square_tool])
# print(xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI))

In [5]:
# create execution
xpander_agent.invoke_agent("What is the square of 14 + 12 squared? Calculate step by step")
xpander_agent.memory.initialize(input=xpander_agent.execution.input_message,instructions=xpander_agent.instructions)

while not xpander_agent.is_finished():
    print("-"*100)
    response = openai_client.chat.completions.create(
                model= 'gpt-4o',
                messages=xpander_agent.memory.retrieve_messages(),
                tools=xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI),
                tool_choice="auto",
                temperature=0.0
        )
    print(response.model_dump())
    # add messages directly from the LLM response
    xpander_agent.memory.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump(),llm_provider=LLMProvider.OPEN_AI)
    # run tools
    xpander_agent.run_tools(tool_calls=tool_calls)

# result (re fetch execution result)
# IMPORTANT: LAST TOOL IS xpfinish-agent-execution-finished WHICH IS AGENT-END with PARSING and may be slower due to inference times
execution_result = xpander_agent.execution.fetch(agent=xpander_agent,execution_id=xpander_agent.execution.id)
print("status", execution_result.status)
print("result", execution_result.result)

----------------------------------------------------------------------------------------------------
{'id': 'chatcmpl-AvBPXADsxwTtYyy96dpXkvXju0M1Z', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'It seems I misunderstood the expression. Let\'s clarify and calculate it correctly:\n\nThe expression "the square of 14 + 12 squared" can be interpreted as:\n\n1. **Calculate 12 squared:**\n   \\[\n   12^2 = 12 \\times 12 = 144\n   \\]\n\n2. **Add 14 to the result of 12 squared:**\n   \\[\n   14 + 144 = 158\n   \\]\n\n3. **Calculate the square of the result from step 2:**\n   \\[\n   158^2 = 158 \\times 158 = 24,964\n   \\]\n\nTherefore, the correct result of the expression "the square of 14 + 12 squared" is 24,964.', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}}], 'created': 1738191923, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'service_tier': 'default', 'system_fingerprint': 'fp_

KeyboardInterrupt: 